In [3]:
#!/usr/bin/env sage
# Standalone GMiMC implementation and polynomial system extractor
# No external file dependencies

class Gmimc:
    '''
    GMiMC cipher implementation for polynomial system extraction
    '''
    def __init__(self, field, exponent, master_key, constants, n, round_keys=None):
        if field.is_field():
            assert field.is_prime_field(), f"This implementation of GMiMC is only defined over prime fields, not over {field}."
        else:
            assert field.is_ring(), f"The parameter 'field' must be either a field or a polynomial ring, not {type(field)}."
            assert field.base_ring().is_prime_field(), f"This implementation of GMiMC is only defined over prime fields, not over {field.base_ring()}."
        assert gcd(field.characteristic(), exponent) == 1, f"For finite field of size {field.characteristic()}, exponent {exponent} is not a permutation."
        assert n > 2, f"GMiMC needs at least 3 branches, not {n}."
        if round_keys:
            assert len(round_keys) <= len(constants), f"The supplied round keys ({len(round_keys)}) imply a greater number of rounds than the constants ({len(constants)})."
            round_keys = [field(k) for k in round_keys]
        master_key = field(master_key)
        constants = [field(c) for c in constants]
        self.field = field
        self.exponent = exponent
        self.__master_key = master_key
        self.constants = constants
        self.n = n
        self.__round_keys = round_keys

    def __call__(self, plaintext, use_supplied_round_keys=True):
        '''
        Encrypt the given plaintext, returning the ciphertext.
        '''
        assert len(plaintext) == self.n, f"Plaintext is of length {len(plaintext)} but should be of length {self.n}."
        ciphertext = [self.field(pt) for pt in plaintext]
        for i in range(len(self.constants)):
            ciphertext = self.feistel_erf(i, ciphertext, use_supplied_round_keys=use_supplied_round_keys)
        return ciphertext

    def feistel_erf(self, i, state, use_supplied_round_keys=True):
        '''
        Applies one round of a generalized Feistel network with expanding round function to state, returning the new state.
        '''
        assert len(state) == self.n, f"ERF input is of length {len(state)} but should be of length {self.n}."
        a = self.f(i, state[0], use_supplied_round_keys=use_supplied_round_keys)
        return [s + a for s in state[1:]] + [state[0]]

    def f(self, i, elem, use_supplied_round_keys=True):
        '''
        Adds up supplied element, ith round key, ith round constant, returning the power of the sum.
        '''
        assert i < len(self.constants), f"Only {len(self.constants)} rounds defined, but round {i} requested."
        assert i >= 0, f"No round function with negative index {i} defined."
        k = self._round_key(i, use_supplied_round_keys=use_supplied_round_keys)
        c = self.constants[i]
        return (elem + c + k)**self.exponent

    def _round_key(self, i, use_supplied_round_keys=True):
        '''
        Returns the ith round key in the key schedule, or optionally the correct supplied round key.
        '''
        round_key = (i + 1) * self.__master_key
        if use_supplied_round_keys and self.__round_keys and i < len(self.__round_keys):
            round_key = self.__round_keys[i]
        return round_key


def extract_gmimc_system(prime, num_rounds, verbose=True):
    """
    Extract polynomial system for GMiMC cipher
    """
    if verbose:
        print(f"=== GMiMC Polynomial System Extraction ===")
        print(f"Prime field: F_{prime}")
        print(f"Number of rounds: {num_rounds}")
    
    # GMiMC parameters
    constants = [11, 80, 55, 33, 67, 23, 45, 89]  # Available constants
    if num_rounds > len(constants):
        print(f"Warning: Only {len(constants)} constants available, using {len(constants)} rounds")
        num_rounds = len(constants)
    
    constants = constants[:num_rounds]
    exponent = 3
    master_key = 42
    n = 3  # Number of branches
    
    # Input plaintext (3 elements for GMiMC)
    plaintext = [0, 1, 2]
    
    if verbose:
        print(f"Constants: {constants}")
        print(f"Exponent: {exponent}")
        print(f"Master key: {master_key}")
        print(f"Number of branches: {n}")
        print(f"Input plaintext: {plaintext}")
    
    # Create polynomial ring with variables for round keys
    ring = PolynomialRing(GF(prime), 'k', num_rounds)
    round_key_vars = ring.gens()
    
    if verbose:
        print(f"Round key variables: {round_key_vars}")
    
    # Create GMiMC instance with symbolic round keys
    gmimc = Gmimc(ring, exponent, master_key, constants, n, round_keys=round_key_vars)
    
    # Get symbolic output (polynomials in terms of round keys)
    symbolic_output = gmimc(plaintext, use_supplied_round_keys=True)
    
    if verbose:
        print(f"\n=== Symbolic Output (Ciphertext as polynomials in round keys) ===")
        for i, poly in enumerate(symbolic_output):
            print(f"c_{i} = {poly}")
        
        print(f"\n=== System Statistics ===")
        degrees = [poly.degree() for poly in symbolic_output]
        print(f"Degrees: {degrees}")
        print(f"Maximum degree: {max(degrees)}")
        print(f"Total monomials: {sum(len(poly.monomials()) for poly in symbolic_output)}")
    
    return symbolic_output, ring, plaintext


def create_attack_system(prime, num_rounds, known_ciphertext=None, verbose=True):
    """
    Create a cryptanalytic system: given known ciphertext, find round keys
    """
    if verbose:
        print(f"\n=== Creating Attack System ===")
    
    # Get symbolic system
    symbolic_output, ring, plaintext = extract_gmimc_system(prime, num_rounds, verbose=False)
    
    # If no known ciphertext provided, compute one for demonstration
    if known_ciphertext is None:
        # Create concrete GMiMC to compute actual ciphertext
        F = GF(prime)
        constants = [11, 80, 55, 33, 67, 23, 45, 89][:num_rounds]
        gmimc_concrete = Gmimc(F, 3, 42, constants, 3)
        known_ciphertext = gmimc_concrete(plaintext)
        if verbose:
            print(f"Computed known ciphertext: {known_ciphertext}")
    
    # Create attack equations: symbolic_output[i] - known_ciphertext[i] = 0
    attack_equations = []
    for i, (symbolic, known) in enumerate(zip(symbolic_output, known_ciphertext)):
        equation = symbolic - known
        attack_equations.append(equation)
        if verbose:
            print(f"Equation {i}: {equation} = 0")
    
    if verbose:
        print(f"\nAttack system: {len(attack_equations)} equations in {len(ring.gens())} unknowns")
        degrees = [eq.degree() for eq in attack_equations]
        print(f"Equation degrees: {degrees}")
        print(f"Maximum degree: {max(degrees)}")
        print(f"Total terms: {sum(len(eq.monomials()) for eq in attack_equations)}")
    
    return attack_equations, ring, known_ciphertext


def solve_attack_system(attack_equations, ring, verbose=True):
    """
    Attempt to solve the attack system using Gröbner basis
    """
    if verbose:
        print(f"\n=== Solving Attack System ===")
        print(f"Computing Gröbner basis...")
    
    try:
        import time
        start_time = time.time()
        
        # Compute Gröbner basis
        ideal = Ideal(attack_equations)
        gb = ideal.groebner_basis()
        
        end_time = time.time()
        
        if verbose:
            print(f"Gröbner basis computed in {end_time - start_time:.3f} seconds")
            print(f"Gröbner basis has {len(gb)} polynomials")
        
        if gb:
            gb_degrees = [p.degree() for p in gb if p != 0]
            if gb_degrees and verbose:
                print(f"Gröbner basis degrees: {gb_degrees}")
                print(f"Maximum degree in GB: {max(gb_degrees)}")
            
            # Check if system is inconsistent (no solutions)
            if len(gb) == 1 and gb[0] == 1:
                if verbose:
                    print("System is inconsistent (no solutions)")
                return None, gb
            
            # Try to find solutions
            if verbose:
                print("Attempting to find variety...")
            
            try:
                variety = ideal.variety()
                if verbose:
                    print(f"Found {len(variety)} solution(s):")
                    for i, sol in enumerate(variety):
                        print(f"  Solution {i}: {sol}")
                return variety, gb
            
            except Exception as e:
                if verbose:
                    print(f"Variety computation failed: {e}")
                return None, gb
        
        else:
            if verbose:
                print("Empty Gröbner basis")
            return None, gb
            
    except Exception as e:
        if verbose:
            print(f"Gröbner basis computation failed: {e}")
        return None, None


def main():
    """
    Main function demonstrating GMiMC polynomial system extraction and attack
    """
    print("GMiMC Polynomial System Extractor")
    print("=" * 50)
    
    # Parameters
    prime = 101  # Small prime for testing
    max_rounds = 3
    
    # Test with increasing number of rounds
    for num_rounds in range(1, max_rounds + 1):
        print(f"\n{'='*20} Testing {num_rounds} rounds {'='*20}")
        
        # Extract polynomial system
        symbolic_output, ring, plaintext = extract_gmimc_system(prime, num_rounds, verbose=True)
        
        # Create attack system
        attack_equations, ring, known_ciphertext = create_attack_system(prime, num_rounds, verbose=True)
        
        # For small systems, try to solve
        if num_rounds <= 3:
            solutions, gb = solve_attack_system(attack_equations, ring, verbose=True)
            
            if solutions:
                print(f"\n=== Attack Successful for {num_rounds} rounds! ===")
                # Verify solution
                if solutions:
                    sol = solutions[0]
                    recovered_keys = [sol.get(ring.gen(i), None) for i in range(num_rounds)]
                    print(f"Recovered round keys: {recovered_keys}")
                    
                    # Expected keys from key schedule
                    expected_keys = [(i + 1) * 42 for i in range(num_rounds)]  
                    expected_keys = [k % prime for k in expected_keys]  # Reduce mod prime
                    print(f"Expected round keys:  {expected_keys}")
        else:
            print(f"\n=== System too large for demonstration solving ===")
            print(f"Polynomial system successfully extracted and ready for analysis")
    
    print(f"\n=== Summary ===")
    print(f"Successfully demonstrated GMiMC polynomial system extraction")
    print(f"Systems can be exported for analysis with specialized solvers")


if __name__ == "__main__":
    main()

GMiMC Polynomial System Extractor

==================== Testing 1 rounds ====================
=== GMiMC Polynomial System Extraction ===
Prime field: F_101
Number of rounds: 1
Constants: [11]
Exponent: 3
Master key: 42
Number of branches: 3
Input plaintext: [0, 1, 2]
Round key variables: (k,)

=== Symbolic Output (Ciphertext as polynomials in round keys) ===
c_0 = k^3 + 33*k^2 - 41*k + 19
c_1 = k^3 + 33*k^2 - 41*k + 20
c_2 = 0

=== System Statistics ===
Degrees: [3, 3, -1]
Maximum degree: 3
Total monomials: 8

=== Creating Attack System ===
Computed known ciphertext: [4, 5, 0]
Equation 0: k^3 + 33*k^2 - 41*k + 15 = 0
Equation 1: k^3 + 33*k^2 - 41*k + 15 = 0
Equation 2: 0 = 0

Attack system: 3 equations in 1 unknowns
Equation degrees: [3, 3, -1]
Maximum degree: 3
Total terms: 8

=== Solving Attack System ===
Computing Gröbner basis...
Gröbner basis computed in 0.001 seconds
Gröbner basis has 1 polynomials
Gröbner basis degrees: [3]
Maximum degree in GB: 3
Attempting to find variety...
F